In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import joblib

In [2]:
# Load the saved EfficientNet model
efficientnet_model_path = r"C:\Users\K M SASTRY\Desktop\PROJECT\efficientnet_model.h5"
efficientnet_model = load_model(efficientnet_model_path)

In [3]:
# Remove the top classification layer to get features
feature_extractor = tf.keras.Model(inputs=efficientnet_model.input, outputs=efficientnet_model.layers[-2].output)

# Prepare the data generator for feature extraction
data_dir = r"K:\PROJECT\ORGANISED DATASET"
img_height, img_width = 224, 224
batch_size = 32

datagen = ImageDataGenerator(rescale=1.0 / 255)

In [4]:
# Extract features for training
train_generator = datagen.flow_from_directory(
    directory=f"{data_dir}/train",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="binary",
    shuffle=False,
)
X_train = feature_extractor.predict(train_generator)
y_train = train_generator.classes

# Extract features for testing
test_generator = datagen.flow_from_directory(
    directory=f"{data_dir}/test",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="binary",
    shuffle=False,
)
X_test = feature_extractor.predict(test_generator)
y_test = test_generator.classes

Found 6185 images belonging to 2 classes.
194/194 [==============================] - 224s 1s/step
Found 1724 images belonging to 2 classes.
54/54 [==============================] - 63s 1s/step


In [5]:
# Reshape the features
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Train the SVM classifier
svm_clf = SVC(kernel="linear", probability=True)
svm_clf.fit(X_train, y_train)

# Evaluate the SVM classifier
y_pred = svm_clf.predict(X_test)

In [6]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[   0  542]
 [   0 1182]]

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       542
           1       0.69      1.00      0.81      1182

    accuracy                           0.69      1724
   macro avg       0.34      0.50      0.41      1724
weighted avg       0.47      0.69      0.56      1724



C:\Users\K M SASTRY\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\K M SASTRY\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\K M SASTRY\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
joblib.dump(svm_clf, r"C:\Users\K M SASTRY\Desktop\PROJECT\eff_svm_classifier.pkl")
print("SVM classifier saved successfully.")

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import joblib

# Load the trained SVM model
svm_clf = joblib.load(r"C:\Users\K M SASTRY\Desktop\PROJECT\eff_svm_classifier.pkl")

# Preprocess the input image
def preprocess_image(image_path, target_size):
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalize the image
    return img_array

# Path to the input image
img_path = r"K:\PROJECT\ORGANISED DATASET\test\BENIGN\fibroadenoma\SOB_B_F_14-14134\400X\SOB_B_F-14-14134-400-021.png"

# Preprocess the image
input_image = preprocess_image(img_path, target_size=(224, 224))

# Extract features using the CNN feature extractor
features = feature_extractor.predict(input_image)
features = features.reshape(1, -1)  # Reshape to fit SVM input

# Make predictions with the SVM
prediction = svm_clf.predict(features)
prediction_proba = svm_clf.predict_proba(features)

# Decode the predicted label
class_indices = train_generator.class_indices
class_labels = {v: k for k, v in class_indices.items()}
predicted_label = class_labels[prediction[0]]

# Format probabilities as percentages
prediction_proba_percent = prediction_proba[0] * 100

# Display the input image and prediction results
plt.imshow(image.load_img(img_path))
plt.axis('off')
plt.title(f"Predicted Class: {predicted_label}\n"
          f"Probabilities: Benign: {prediction_proba_percent[0]:.2f}%, "
          f"Malignant: {prediction_proba_percent[1]:.2f}%")
plt.show()
